In [1]:
import pandas as pd
from ddparser import DDParser
from collections import Counter


In [2]:
# baido module
ddp = DDParser(use_pos=True)

2020-12-28 15:09:23,249-INFO: loading the fields.


In [7]:
#####testing area
text="他撥弄她的頭髮，時而捲動，時而打成一個結。"
test=ddp.parse(text)[0]
syntax_dict=dict(zip([x+1 for x in range(len(test["word"]))],list(zip(test["word"],test["head"],test["deprel"],test["postag"]))))

#output:
#   {1: ('他', 2, 'SBV', 'r'),
#    2: ('撥弄', 0, 'HED', 'v'),
#    3: ('她', 5, 'ATT', 'r'),
#    4: ('的', 3, 'MT', 'u'),
#    5: ('頭髮', 2, 'VOB', 'n'),
#    6: ('，', 2, 'MT', 'w'),
#    7: ('時而', 8, 'ADV', 'd'),
#    8: ('捲動', 2, 'COO', 'v'),
#    9: ('，', 8, 'MT', 'w'),
#    10: ('時而', 11, 'ADV', 'd'),
#    11: ('打成', 2, 'COO', 'v'),
#    12: ('一個', 13, 'ATT', 'm'),
#    13: ('結', 11, 'VOB', 'n'),
#    14: ('。', 11, 'MT', 'w')}

In [9]:
def syntax_analyze(string):
    ### step1 : create syntax dict from baido result

    result=ddp.parse(string)[0]
    syntax_dict=dict(zip([x+1 for x in range(len(result["word"]))],list(zip(result["word"],result["head"],result["deprel"],result["postag"]))))

    ### step2 : find subject

        #find word if head=="SBV" and deprel=="PER" or "r"
    s_index_list=[key for key,value in syntax_dict.items() if value[2]=="SBV" and value[3] in ["PER","r","ORG"]]

    ### step3 : find verb, which pointed by subject

        #pick verb word and verb word's VV or COO
    s_v_dict={}
    for s_index in s_index_list:
        #get index pointed by subject 
        v_index=syntax_dict[s_index][1]
        v_index_list=[]

        #add word to verb list if the pointed index is verb
        if syntax_dict[v_index][3]=="v":
            v_index_list=[v_index]

        #add word to verb list if verb have other verb appositive(同位語動詞)
        v_index_list=v_index_list+[key for key,value in syntax_dict.items() if value[2] in ["VV","COO"] and value[1] == v_index and value[3]=="v" ]

        # verb_index_list could be zero because subject's head cound be non verb and have no "VV" or "COO"
        if len(v_index_list)>0:
            s_v_dict.update({s_index:v_index_list})

    ### step4 : find object of each verb

    s_v_o_dict={}
    for s_index,v_list in s_v_dict.items():
        verb_object_list=[]
        for verb_index in v_list:
            object_index=None

            #find any object point to the verb
            for syn_index,syn_value in syntax_dict.items():
                # "VOB" is object, syn_value[1] is points index, syn_value[2] is part of speech
                if syn_value[1]==verb_index and syn_value[2]=="VOB":
                    object_index=syn_index
            verb_object_list=verb_object_list+[(verb_index,object_index)]
        s_v_o_dict.update({s_index:verb_object_list})

    # step5 : convert result index to word
    result_text_dict={}
    for key,value in s_v_o_dict.items():
        key_text=syntax_dict[key][0]
        v_o_tuple_list_text=[]
        for v_o_tuple in value:
            index_v=v_o_tuple[0]
            index_o=v_o_tuple[1]
            if index_o is not None:
                v_o_tuple_text=(syntax_dict[index_v][0],syntax_dict[index_o][0])
            else:
                v_o_tuple_text=(syntax_dict[index_v][0],"")
            v_o_tuple_list_text+=[v_o_tuple_text]
        result_text_dict.update({key_text:v_o_tuple_list_text})
    return result_text_dict


In [10]:
syntax_analyze(text)

{'他': [('撥弄', '頭髮'), ('捲動', ''), ('打成', '結')]}